In [1]:
from bloqade import rydberg_h, piecewise_linear, piecewise_constant, waveform, cast, save, load
from bloqade.atom_arrangement import ListOfLocations, Lieb, Square, Chain, Honeycomb, Kagome, Triangular, Rectangular
from bokeh.io import output_notebook # to plot "show()" on the notebook, without opening a browser
import os
import matplotlib.pyplot as plt
import numpy as np

output_notebook()

Loading BokehJS ...

In [23]:
# Load delta data from CSV
delta_values = np.genfromtxt("/home/jovyan/2024_Quera/delta_values.csv", delimiter=", ", usemask=False)
omega_values = np.genfromtxt("/home/jovyan/2024_Quera/omega_values.csv", delimiter=", ", usemask=False)
delta_values = delta_values * (6.8 / max(delta_values)) # scale delta values by 2pi
omega_values = omega_values * (2.1 / max(omega_values)) # scale omega values by 2pi

In [36]:
# geometry
rng = np.random.default_rng(1234)
atom_pos=Square(4, lattice_spacing=4).apply_defect_density(0.2, rng=rng).remove_vacant_sites()

#dynamics
durations = np.diff(np.arange(0.01, 3.6, 0.06)).tolist()
delta_MHz = [delta_values[10*i] for i in range(60)]
omega_MHz = [omega_values[10*i] for i in range(60)]
omega_MHz[0] = 0
omega_MHz[-1] = 0

Delta = piecewise_linear(durations, [x*2*np.pi for x in delta_MHz])
Omega = piecewise_linear(durations, [x*2*np.pi for x in omega_MHz])

#create Hamiltonian
program = rydberg_h(atom_pos, detuning = Delta, amplitude = Omega, phase = None)

program.parse_register().show()
program.parse_sequence().show()

In [18]:
# Test the program with a classical simulation
output = program.bloqade.python().run(shots=100, interaction_picture=True)

In [19]:
# Print bitstrings in order of decreasing probability
output.report().bitstrings()

[array([[0, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 0, 1],
        ...,
        [1, 0, 1, ..., 1, 0, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 0, 1, ..., 1, 0, 1]], dtype=int8)]

In [40]:
# Submit to quantum hardware
output_Q = program.braket.aquila().run_async(shots=100)

In [48]:
# Retrieve 
output_Q.fetch()

                                             task ID     status  shots
0  arn:aws:braket:us-east-1:592242689881:quantum-...  Completed    100

In [49]:
# Save results
number = 1
save(output_Q, f"output_Q_{number}")
new_output = load(f"output_Q_{number}")

In [51]:
# Print bitstrings in order of decreasing probability
new_output.report().bitstrings()

[array([[0, 1, 1, ..., 1, 1, 0],
        [0, 1, 0, ..., 1, 1, 0],
        [0, 1, 1, ..., 1, 1, 0],
        ...,
        [0, 1, 1, ..., 1, 1, 0],
        [1, 0, 1, ..., 1, 1, 0],
        [0, 1, 1, ..., 1, 1, 0]], dtype=int8)]

In [55]:
# Print probabilities of each bitstring outcome
output.report().counts()

[OrderedDict([('011011111110110', 5),
              ('011011101111110', 4),
              ('011011111111110', 3),
              ('011111011110110', 3),
              ('101011111010111', 3),
              ('111001111010111', 3),
              ('010111101111110', 2),
              ('011011110111110', 2),
              ('011011111110111', 2),
              ('011011111111011', 2),
              ('011111101111110', 2),
              ('101111001111110', 2),
              ('101111111110110', 2),
              ('111011111110101', 2),
              ('111011111110110', 2),
              ('111100111110101', 2),
              ('111101111110101', 2),
              ('010111101101111', 1),
              ('010111111100111', 1),
              ('010111111101111', 1),
              ('010111111110110', 1),
              ('010111111111010', 1),
              ('010111111111110', 1),
              ('011011110110111', 1),
              ('011011110111111', 1),
              ('011011111010111', 1),
            

In [53]:
# Print Rydberg densities
output.report().rydberg_densities()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
task_number,,,,,,,,,,,,,,,
0,0.46,0.22,0.12,0.54,0.18,0.07,0.15,0.17,0.07,0.1,0.15,0.47,0.16,0.16,0.48


In [56]:
# Generate graphs
output.report().show()